In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()
if True:
  llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
else:
  llm = ChatGroq(model="llama-3.3-70b-versatile")

result = llm.invoke("Sing a ballad of LangChain.")
print(result.content)

(Verse 1)
The digital world, a boundless sea,
Where models spoke with majesty.
But isolated, vast, and deep,
Their grandest thoughts they could not keep.
No memory held, no tool to wield,
A lonely power, unrevealed.

(Verse 2)
Then from the code, a legend grew,
A framework robust, and ever new.
'Twas LangChain named, with purpose clear,
To banish doubt, and quell all fear.
A weaver of logic, strong and true,
To make the mighty models new.

(Verse 3)
The LLM, its heart so bright,
Would answer questions, day and night.
But LangChain taught it how to learn,
A sequence grand, at every turn.
With structured prompts, both sharp and keen,
The model's wisdom could be seen.

(Verse 4)
No longer lost in fleeting thought,
Its memory by LangChain brought.
A history kept, a context deep,
Secrets that the models would keep.
And 'Retrievers,' swift and ever bold,
New knowledge from the world unfold.

(Verse 5)
Then 'Tools' it grasped, with eager hand,
To search the web, or understand.
To calculate, t

In [2]:
from langchain.agents import initialize_agent, AgentType, tool
from langchain_tavily import TavilySearch
import datetime

@tool
def get_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
  """Return the current date and time in this format."""
  return datetime.datetime.now().strftime(format)


tools = [TavilySearch(max_results=2), get_system_time]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # structured agent
    verbose=True
)

agent.invoke("Give me a funny tweet about today's weather in Bhind.")


C:\Users\Daksh\AppData\Local\Temp\ipykernel_8320\624674990.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "tavily_search",
  "action_input": {
    "query": "today's weather in Bhind"
  }
}
```
Observation: {'query': "today's weather in Bhind", 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Bhind', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Bhind', 'region': 'Madhya Pradesh', 'country': 'India', 'lat': 26.5642, 'lon': 78.7883, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1758283629, 'localtime': '2025-09-19 17:37'}, 'current': {'last_updated_epoch': 1758283200, 'last_updated': '2025-09-19 17:30', 'temp_c': 31.5, 'temp_f': 88.7, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 6.5, 'wind_kph': 10.4, 'wind_degree': 27, 'wind_dir': 'NNE', 'pressure_mb': 1003.0, 'pressure_in': 29.62, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 62, 'cloud': 17, 'feelslike_c': 36.6, 'feelslik

{'input': "Give me a funny tweet about today's weather in Bhind.",
 'output': "Bhind's weather today is 31.5°C but feels like a scorching 36.6°C! The sun isn't just shining, it's actively trying to turn us all into perfectly toasted papads. Send water, or better yet, an AC! ☀️🔥 #BhindWeather #TooHotToHandle"}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generation_prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      " you are a twitter techie influencer assistant tasked with writing excellent twitter posts."
      " Generate the best twitter post possible for the user's request."
      " If the user provides critique, respond with revised version of your previous attempts."
    ),
    MessagesPlaceholder(variable_name="messages"),
  ]
)

reflection_prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      " You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
      " Always provides detailed recommendations, including requests for length, virality, style, etc."
    ),
    MessagesPlaceholder(variable_name="messages"),
  ]
)

generation_agent = generation_prompt | llm
reflection_agent = reflection_prompt | llm

In [14]:
from typing import List, Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph

graph = MessageGraph()

REFLECT = "reflect"
GENERATE = "generate"

def generate_node(state):
  return generation_agent.invoke({
    "messages": state
  })


def reflect_node(state):
  response = reflection_agent.invoke({
    "message": state
  })
  return [HumanMessage(content=response.content)]

graph.add_node(GENERATE, generate_node)
graph.add_node(REFLECT, reflect_node)

graph.set_entry_point(GENERATE)

def should_continue(state):
  if (len(state) > 4):
    return END
  
  return REFLECT

graph.add_conditional_edges(GENERATE, should_continue)
graph.add_edge(REFLECT, GENERATE)

app = graph.compile()

print(app.get_graph().draw_mermaid())
app.get_graph().print_ascii()

C:\Users\Daksh\AppData\Local\Temp\ipykernel_8320\3151997862.py:5: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = MessageGraph()


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	generate(generate)
	reflect(reflect)
	__end__(<p>__end__</p>)
	__start__ --> generate;
	generate --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



ImportError: Install grandalf to draw graphs: `pip install grandalf`.